In [25]:
HTML(read(open("style.css"), String))

HTML{String}("<link href='http://fonts.googleapis.com/css?family=Alegreya+Sans:100,300,400,500,700,800,900,100italic,300italic,400italic,500italic,700italic,800italic,900italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Arvo:400,700,400italic' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=PT+Mono' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Shadows+Into+Light' rel='stylesheet' type='text/css'>\r\n<link href='http://fonts.googleapis.com/css?family=Philosopher:400,700,400italic,700italic' rel='stylesheet' type='text/css'>\r\n\r\n<style>\r\n\r\n@font-face {\r\n    font-family: \"Computer Modern\";\r\n    src: url('http://mirrors.ctan.org/fonts/cm-unicode/fonts/otf/cmunss.otf');\r\n}\r\n\r\n.container { width: 100% }\r\n\r\n/* Formatting for header cells */\r\n.text_cell_render h1 {\r\n    font-family: 'Philosopher', sans-serif;\r\n    font-weight: 400;\r\n    font-size: 2.2em;\r\n    line-height: 100%;\r\n    color: rgb(0, 80, 120);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n.text_cell_render h2 {\r\n    font-family: 'Philosopher', serif;\r\n    font-weight: 400;\r\n    font-size: 1.9em;\r\n    line-height: 100%;\r\n    color: rgb(200,100,0);\r\n    margin-bottom: 0.1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\t\r\n\r\n.text_cell_render h3 {\r\n    font-family: 'Philosopher', serif;\r\n    margin-top:12px;\r\n    margin-bottom: 3px;\r\n    font-style: italic;\r\n    color: rgb(94,127,192);\r\n}\r\n\r\n.text_cell_render h4 {\r\n    font-family: 'Philosopher', serif;\r\n}\r\n\r\n.text_cell_render h5 {\r\n    font-family: 'Alegreya Sans', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 16pt;\r\n    color: grey;\r\n    font-style: italic;\r\n    margin-bottom: .1em;\r\n    margin-top: 0.1em;\r\n    display: block;\r\n}\r\n\r\n.text_cell_render h6 {\r\n    font-family: 'PT Mono', sans-serif;\r\n    font-weight: 300;\r\n    font-size: 10pt;\r\n    color: grey;\r\n    margin-bottom: 1px;\r\n    margin-top: 1px;\r\n}\r\n\r\n.text_cell_render em {\r\n    font-family: 'Philosopher', sans-serif;\r\n    color:        blue;\r\n    background-color: rgb(255,220,180);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   100;\r\n}\r\n\r\n.text_cell_render b {\r\n    color:            rgb(255,195,195);\r\n    background-color: rgb(0,0,0);\r\n    font-size:    110%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   650;\r\n}\r\n\r\n.text_cell_render tt {\r\n    font-size:    120%;\r\n    margin-left:   2px;\r\n    margin-right:  2px;\r\n    font-weight:   150;\r\n}\r\n\r\n.Codemirror {\r\n    font-family: \"PT Mono\";\r\n    font-size: 100%;\r\n}\r\n\r\n</style>\r\n\r\n")

# Play a Chess Game

In [26]:
using Pkg
# Pkg.add("Chess")
using Chess
using Random

# Pkg.add("NBInclude")
using NBInclude

In [27]:
@nbinclude("Chess Games Save.ipynb")

LoadError: LoadError: MethodError: objects of type Board are not callable
in expression starting at C:\Users\yiwen\Documents\GitHub\ChessAIJulia\Julia\Random Chess.ipynb:In[7]:2
in expression starting at C:\Users\yiwen\Documents\GitHub\ChessAIJulia\Julia\Chess Games Save.ipynb:In[3]:1

In [28]:
@nbinclude("ValidateUserInput.ipynb")

false

## Side functions

`printGame` function prints the board to standard output. Settings can be set here. 

In [ ]:
function printGame(game)
    pprint(board(game), color = true)
end

`printWin(game)` prints the result of the game as a text message.

In [ ]:
function printWin(board)
    if ischeckmate(board)
        color = sidetomove(board)
        color == "WHITE" ? color = "BLACK" : color = "WHITE"
        print(color * " wins by checkmate" )
    elseif isstalemate(board)
        print("Stalemate")
    elseif ismaterialdraw(board)
        print("Draw by insufficient material")
    elseif isrule50draw(board)
        print("Draw by 50 move draw")
    end
end

## Play game

In this section you can play a chess game against one of the programmed "engines". In `playMove` you can specify which algorithm you want to play against. Before that you have to load in the nessesary Jupyter Notebook with the algorithm in it.

In [ ]:
@nbinclude("Random Chess.ipynb")

In [ ]:
# @nbinclude("Minimax.ipynb")

### Select your Engine here

In [ ]:
function playMove(game)
    playRandomMove(game)
    # playMinimaxMove(game, 4)
end

In [ ]:
function playGame(color, seed)
    Random.seed!(seed)
    # validation
    game = SimpleGame()
    printGame(game)
    setGameHeaders(game)
    
    # Offset move if player is black
    if color == "black"
        playMove(game)
        printGame(game)
    elseif color != "white"
        print("Invalid color. Choose white or black")   
        return
    end

    while true
        # User Move
        if !isterminal(game)
            # User input validation
            while true
                userMove = readline()
                bool_legal_move = is_legal_move(board(game), movefromstring(userMove))
                if !bool_legal_move
                    println("Invalid Move: "* userMove)
                end
                !bool_legal_move || break
            end
            
            domove!(game, userMove)
            # IJulia.clear_output(true)
            # sleep(1)
            printGame(game)
        else
            break
        end
        
        # Delay
        sleep(1)
        
        # Engine Move
        if !isterminal(game)
            playMove(game)
            # IJulia.clear_output(true)
            # sleep(1)
            printGame(game)
        else
            break
        end
    end
    saveGameToPGN(game)
    printWin(board(game))
    return game
end

In [ ]:
playGame("white", 1233)

In [ ]:
g = SimpleGame()
board(g)